In [1]:
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd
from datetime import datetime
import numpy as np
import time

In [ ]:
URL1 = 'https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Locate/ImageServer/exportImage?f=image&bbox='

In [ ]:
# ---------------------------------------------------------------
# Downloads images in a grid pattern from a given starting point.
#----------------------------------------------------------------


#converts meters to Latitude-Longitude
ImageStep = float(100/111139)
ImageSize = 2000

# starting point
LatRef = 121.507731
LongRef = -30.767985

Iterations = 2
file_type = 'png'

# Enter download folder path
path = '/Mining/'
os.chdir(path)
schema = {'ID' :  str,
         'File_Name' : str,
         'Download_Date_Time' : str,
         'Latitude_1' : float,
         'Longitude_1' : float,
         'Latitude_2' : float,
         'Longitude_2' : float,
         'Format' : str,
         'Size': int,
         'Step':float}
image_df = pd.DataFrame(columns=schema).astype(schema)

for long in range(Iterations):
    for lat in range(Iterations):
        image_number = long*Iterations+lat+1
        Lat1 = LatRef + lat*ImageStep
        Long1 = LongRef + long*ImageStep
        Lat2 = Lat1 + ImageStep
        Long2 = Long1 + ImageStep
        #URL2 = 'bbox='+ Lat1 + ''%2C' + Long1 + %2C' + Lat2 + ''%2C' + Long2
        URL = (URL1 + str(Lat1) + ","+ str(Long1) + ","+ str(Lat2) + "," + str(Long2))
             # + '&format='+file_type +'&size=' + str(ImageSize) + ',' + str(ImageSize))
        print(URL)
        print("Image:"+ str(image_number) + '/' + str(Iterations*Iterations) + "  " + str(Lat1) + ","+ str(Long1) + ","+ str(Lat2) + ","+ str(Long2))

        # Get image from API
        #response = requests.get(URL)
        
        #convert repsonse from binary to an image
       # img = Image.open(BytesIO(response.content))
        
        image_id =datetime.now().strftime("%y%m%d%H%M%S")+str(image_number) 
        filename = image_id +'.'+ file_type
               
        temp = pd.Series({'ID' :  image_id,
                             'File_Name' : filename,
                             'Download_Date_Time' : datetime.now().strftime("%y-%m-%d %H:%M:%S"),
                             'Latitude_1' : Lat1,
                             'Longitude_1' : Long1,
                             'Latitude_2' : Lat2,
                             'Longitude_2' : Long2,
                             'Format' : file_type,
                             'Size': ImageSize,
                             'Step': ImageStep})
        print(temp)  
        image_df = pd.concat([image_df,temp.to_frame().T], ignore_index=True)
        
        #write image to file
        #img.save(filename)
        
display(image_df)

In [ ]:
# ---------------------------------------------------------------
# load abandoned mine data
#----------------------------------------------------------------
am_path = '/Sample Data/'
am_filename = 'wabmines_GDA2020.csv'
am_labels = pd.read_csv(am_path+am_filename)
# display(am_labels)
am_labels2 = am_labels[['SITE_CODE','SITE_TYPE','SITE_SUB_T','LATITUDE','LONGITUDE']].copy()
am_labels2['Stage'] = 'Abandoned'
am_labels2['Stage2'] = 'Abandoned'

display(am_labels2)
am_labels2  = am_labels2.rename(columns =  {  'SITE_CODE':	'SiteCode',
                                            'SITE_TYPE': 'Type',
                                            'SITE_SUB_T': 'SubType',
                                            'LATITUDE': 'Latitude',
                                            'LONGITUDE':'Longitude'})
am_labels2 = am_labels2[['SiteCode','Type','SubType','Stage','Latitude','Longitude','Stage2']]
print(am_labels2.columns)

C:\Users\afrob\AppData\Local\Temp\ipykernel_26600\2153983425.py:6: DtypeWarning: Columns (11,12,13,33) have mixed types. Specify dtype option on import or set low_memory=False.
  am_labels = pd.read_csv(am_path+am_filename)


In [ ]:
# load active mine data

act_path = '/Sample Data/'
act_filename = 'Sites.csv'
act_labels = pd.read_csv(act_path+act_filename)
# display(act_labels)
act_labels = act_labels[['SiteCode','Type','SubType','Stage','Latitude','Longitude']]
act_labels['Stage2'] = 'Active'
display(act_labels)

,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2
0,S0000001,Mine,Openpit,Care and Maintenance,-28.565078,122.480754,Active
1,S0000002,Deposit,Unspecified,Undeveloped,-26.787357,118.574415,Active
2,S0000002,Deposit,Unspecified,Undeveloped,-26.787357,118.574415,Active
3,S0000005,Mine,Openpit,Shut,-29.690617,121.012812,Active
4,S0000006,Mine,Openpit,Shut,-30.239138,121.222793,Active
...,...,...,...,...,...,...,...
47090,S0244127,Mine,Openpit,Shut,-30.380134,121.054678,Active
47091,S0244128,Infrastructure,Dump (waste),Shut,-30.378254,121.056179,Active
47092,S0244129,Infrastructure,Dump (waste),Shut,-30.380064,121.058075,Active
47093,S0244130,Prospect,Drillhole,Undeveloped,-28.675087,117.610910,Active


In [ ]:
# join active and abandoned mine data

MineLabels = pd.concat((am_labels2,act_labels))
MineLabels = MineLabels.reset_index(drop=True)
display(MineLabels)


,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2
0,S0130958,Other,Unspecified,Abandoned,-26.629076,120.238380,Abandoned
1,S0130959,Other,Unspecified,Abandoned,-26.629406,120.238060,Abandoned
2,S0130960,Mine,Shaft,Abandoned,-27.428387,117.866530,Abandoned
3,S0130961,Other,Unspecified,Abandoned,-26.629566,120.238050,Abandoned
4,S0130962,Other,Unspecified,Abandoned,-26.629706,120.238250,Abandoned
...,...,...,...,...,...,...,...
239613,S0244127,Mine,Openpit,Shut,-30.380134,121.054678,Active
239614,S0244128,Infrastructure,Dump (waste),Shut,-30.378254,121.056179,Active
239615,S0244129,Infrastructure,Dump (waste),Shut,-30.380064,121.058075,Active
239616,S0244130,Prospect,Drillhole,Undeveloped,-28.675087,117.610910,Active


In [ ]:
# download mine images centered on coordinates in database files

#converts meters to Latitude-Longitude
ImageStep = float(500/111139)
ImageSize =500
image_number = 0
# starting point


Iterations = 500
file_type = 'png'

# Enter download folder path
path = '/Mining/'
os.chdir(path)

#define df for record keeping
schema = {'ID' :  str,
          'Site_Code': str,
         'File_Name' : str,
         'Download_Date_Time' : str,
         'Latitude_Reference' : float,
         'Longtitude_Reference': float,
         'Latitude_1' : float,
         'Longitude_1' : float,
         'Latitude_2' : float,
         'Longitude_2' : float,
         'Format' : str,
         'Size': int,
         'Step':float}
image_df = pd.DataFrame(columns=schema).astype(schema)

for row in am_labels_major[0:Iterations].itertuples(): 
        #print(row)
        image_number += 1    
        LatRef = float(row[7])
        LongRef = float(row[8])
       

        
        Lat1 = LatRef - ImageStep/2
        Long1 = LongRef - ImageStep/2
        Lat2 = LatRef + ImageStep/2
        Long2 = LongRef + ImageStep/2
        #URL2 = 'bbox='+ Lat1 + ''%2C' + Long1 + %2C' + Lat2 + ''%2C' + Long2
        URL = (URL1 + str(Long1) + ","+ str(Lat1) + ","+ str(Long2) + "," + str(Lat2)
              + '&format='+file_type +'&size=' + str(ImageSize) + ',' + str(ImageSize) + 'compressionQuality=100')

        print(URL)
        print("Image:"+ str(image_number) + '/' + str(Iterations) + "  "+ str(LatRef)+ ','+str(LongRef)+ ','+ str(Lat1) + ","+ str(Long1) + ","+ str(Lat2) + ","+ str(Long2))

        # Get image from API
        response = requests.get(URL)
        #print(response.text)

        #convert repsonse from binary to an image
        img = Image.open(BytesIO(response.content))

        image_id =datetime.now().strftime("%y%m%d%H%M%S")+str(image_number) 
        filename = image_id +'.'+ file_type

        temp = pd.Series({'ID' :  image_id,
                         'Site_Code': row[0],
                         'File_Name' : filename,
                         'Download_Date_Time' : datetime.now().strftime("%y-%m-%d %H:%M:%S"),
                         'Latitude_Reference' : LatRef,
                         'Longtitude_Reference': LongRef,
                         'Latitude_1' : Lat1,
                         'Longitude_1' : Long1,
                         'Latitude_2' : Lat2,
                         'Longitude_2' : Long2,
                         'Format' : file_type,
                         'Size': ImageSize,
                         'Step': ImageStep})
        #print(temp)  
        image_df = pd.concat([image_df,temp.to_frame().T], ignore_index=True)

        #write image to file
        img.save(filename)

#display(image_df)
image_df.to_csv(datetime.now().strftime("%y%m%d%H%M%S")+".csv")

In [ ]:
####
####  Download a single image based on lat and long
####

#converts meters to Latitude-Longitude
ImageStep = float(500/111139)
ImageSize =500
image_number = 0
# starting point
# LatRef = float(row[7])
# LongRef = float(row[8])
       

Iterations = 500
file_type = 'png'

# Enter download folder path
path = '/Mining/'
os.chdir(path)

Lat1 = -36
Long1 = 118
Lat2 = -13
Long2 = 129

# Lat1 = LatRef - ImageStep/2
# Long1 = LongRef - ImageStep/2
# Lat2 = LatRef + ImageStep/2
# Long2 = LongRef + ImageStep/2
URL1 = 'https://services.ga.gov.au/gis/rest/services/NationalBaseMap/MapServer/export?bbox='

# https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Locate/MapServer/export?bbox=121.50666%2C-30.76330%2C121.50869%2C-30.76455
#URL2 = 'bbox='+ Lat1 + ''%2C' + Long1 + %2C' + Lat2 + ''%2C' + Long2
URL = (URL1 + str(Long1) + ","+ str(Lat1) + ","+ str(Long2) + "," + str(Lat2))
        # + '&format='+file_type +'&size=' + str(ImageSize) + ',' + str(ImageSiz?e) + 'compressionQuality=100')

print(URL)
# print("Image:"+ str(image_number) + '/' + str(Iterations) + "  "+ str(LatRef)+ ','+str(LongRef)+ ','+ str(Lat1) + ","+ str(Long1) + ","+ str(Lat2) + ","+ str(Long2))

# Get image from API
response = requests.get(URL)
print(response.text)

#convert repsonse from binary to an image
img = Image.open(BytesIO(response.content))

image_id =datetime.now().strftime("%y%m%d%H%M%S")+str(image_number) 
filename = image_id +'.'+ file_type

#write image to file
img.save(filename)


In [ ]:
# filter data based on 5 mine types

path = '/Mining/'

os.chdir(path)
# print(MineLabels)

selection = ['Shaft','Dump (waste)','Dam','Building','Costean/Trench']

select_df = pd.DataFrame()
i=6
for select in selection:
    
    temp_select =  MineLabels[(MineLabels["SubType"]== select) & (MineLabels["Stage2"]== 'Active')]
    print(temp_select.shape)
    # select_random = np.random.choice(np.arange(0,temp_select.shape[0]),1500,replace=False)
    # temp_select = temp_select.iloc[select_random,:]
    temp_select['MineTypeID'] = i
    select_df = pd.concat((select_df,temp_select))
    i+=1
    # print(select_random)
                          
# for select in selection:

#     temp_select =  MineLabels[(MineLabels["SubType"]== select) & (MineLabels["Stage2"]== 'Active')]
#     print(temp_select.shape)
#     # select_random = np.random.choice(np.arange(0,temp_select.shape[0]),1500,replace=False)
#     # temp_select = temp_select.iloc[select_random,:]
#     # select_df = pd.concat((select_df,temp_select))
#     # # print(select_random)

select_df['Old_Index'] = select_df.index
select_df = select_df.reset_index(drop=True)

(1152, 19)
(1147, 19)
(47, 19)
(256, 19)
(304, 19)


C:\Users\afrob\AppData\Local\Temp\ipykernel_37984\4227530939.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_select['MineTypeID'] = i
C:\Users\afrob\AppData\Local\Temp\ipykernel_37984\4227530939.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_select['MineTypeID'] = i
C:\Users\afrob\AppData\Local\Temp\ipykernel_37984\4227530939.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [ ]:
# fiter data for code testing

# display(select_df)
# print(np.arange(7501,7501+len(select_df)))
# select_df = select_df.set_index(np.arange(7501,7501+len(select_df)))
# display(select_df)

,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2,LatStart,LongStart,LatEnd,...,x_center,y_center,x_center_norm,y_center_norm,width,height,width_norm,height_norm,MineTypeID,Old_Index
0,S0000024,Mine,Shaft,Shut,-27.479588,119.169510,Active,-27.481837,119.167261,-27.477338,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192545
1,S0000043,Mine,Shaft,Care and Maintenance,-29.448376,121.122011,Active,-29.450625,121.119762,-29.446126,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192564
2,S0000095,Mine,Shaft,Shut,-32.201185,121.801211,Active,-32.203435,121.798961,-32.198936,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192610
3,S0000119,Mine,Shaft,Shut,-21.235277,119.724711,Active,-21.237527,119.722461,-21.233028,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192633
4,S0000162,Mine,Shaft,Shut,-29.764777,122.477007,Active,-29.767026,122.474757,-29.762527,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2901,S0239129,Prospect,Costean/Trench,Undeveloped,-23.267051,115.755533,Active,-23.269301,115.753283,-23.264802,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,235231
2902,S0239466,Mine,Costean/Trench,Proposed,-29.785916,122.498118,Active,-29.788165,122.495869,-29.783666,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,235534
2903,S0240504,Prospect,Costean/Trench,Undeveloped,-31.042364,121.058930,Active,-31.044614,121.056681,-31.040115,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,236482
2904,S0240505,Prospect,Costean/Trench,Undeveloped,NaN,NaN,Active,NaN,NaN,NaN,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,236483


[ 7501  7502  7503 ... 10404 10405 10406]


,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2,LatStart,LongStart,LatEnd,...,x_center,y_center,x_center_norm,y_center_norm,width,height,width_norm,height_norm,MineTypeID,Old_Index
7501,S0000024,Mine,Shaft,Shut,-27.479588,119.169510,Active,-27.481837,119.167261,-27.477338,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192545
7502,S0000043,Mine,Shaft,Care and Maintenance,-29.448376,121.122011,Active,-29.450625,121.119762,-29.446126,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192564
7503,S0000095,Mine,Shaft,Shut,-32.201185,121.801211,Active,-32.203435,121.798961,-32.198936,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192610
7504,S0000119,Mine,Shaft,Shut,-21.235277,119.724711,Active,-21.237527,119.722461,-21.233028,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192633
7505,S0000162,Mine,Shaft,Shut,-29.764777,122.477007,Active,-29.767026,122.474757,-29.762527,...,250,250,0.5,0.5,50.0,50.0,0.10,0.10,6,192664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10402,S0239129,Prospect,Costean/Trench,Undeveloped,-23.267051,115.755533,Active,-23.269301,115.753283,-23.264802,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,235231
10403,S0239466,Mine,Costean/Trench,Proposed,-29.785916,122.498118,Active,-29.788165,122.495869,-29.783666,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,235534
10404,S0240504,Prospect,Costean/Trench,Undeveloped,-31.042364,121.058930,Active,-31.044614,121.056681,-31.040115,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,236482
10405,S0240505,Prospect,Costean/Trench,Undeveloped,NaN,NaN,Active,NaN,NaN,NaN,...,250,250,0.5,0.5,90.0,90.0,0.18,0.18,10,236483


In [ ]:
#Load No mine data

non_mine = pd.read_csv("Non_Mine.csv", header=None)
display(non_mine.shape)
non_mine2 = non_mine[non_mine[4] == 0].copy()
non_mine2 = non_mine2[non_mine2[1] <=123]
non_mine2 = non_mine2.reset_index(drop=True)
print(non_mine2)
select_random = np.random.choice(np.arange(0,non_mine2.shape[0]),1500,replace=False)
non_mine2 = non_mine2.iloc[select_random,:]
display(non_mine2)

non_mine2 = non_mine2.set_index(np.arange(10407,10407+len(non_mine2)))

print(non_mine2)

(4937284, 6)

                 0           1          2           3    4    5
0       -31.000000  118.000000 -30.995501  118.004499  0.0  0.0
1       -30.995501  118.000000 -30.991002  118.004499  0.0  0.0
2       -30.991002  118.000000 -30.986503  118.004499  0.0  0.0
3       -30.986503  118.000000 -30.982005  118.004499  0.0  0.0
4       -30.982005  118.000000 -30.977506  118.004499  0.0  0.0
...            ...         ...        ...         ...  ...  ...
2425018 -21.026003  122.998245 -21.021505  123.002744  0.0  0.0
2425019 -21.021505  122.998245 -21.017006  123.002744  0.0  0.0
2425020 -21.017006  122.998245 -21.012507  123.002744  0.0  0.0
2425021 -21.012507  122.998245 -21.008008  123.002744  0.0  0.0
2425022 -21.008008  122.998245 -21.003509  123.002744  0.0  0.0

[2425023 rows x 6 columns]


,0,1,2,3,4,5
1316640,-23.005507,120.708320,-23.001008,120.712819,0.0,0.0
1827490,-25.790308,121.774553,-25.785809,121.779051,0.0,0.0
2326793,-23.873789,122.800295,-23.869290,122.804794,0.0,0.0
160245,-21.696335,118.323919,-21.691836,118.328418,0.0,0.0
972070,-23.509380,119.997499,-23.504881,120.001997,0.0,0.0
...,...,...,...,...,...,...
1873541,-24.094233,121.869029,-24.089734,121.873528,0.0,0.0
250921,-26.555116,118.512871,-26.550617,118.517370,0.0,0.0
234561,-21.435401,118.476880,-21.430902,118.481379,0.0,0.0
430557,-25.317926,118.881779,-25.313427,118.886278,0.0,0.0


               0           1          2           3    4    5
10407 -23.005507  120.708320 -23.001008  120.712819  0.0  0.0
10408 -25.790308  121.774553 -25.785809  121.779051  0.0  0.0
10409 -23.873789  122.800295 -23.869290  122.804794  0.0  0.0
10410 -21.696335  118.323919 -21.691836  118.328418  0.0  0.0
10411 -23.509380  119.997499 -23.504881  120.001997  0.0  0.0
...          ...         ...        ...         ...  ...  ...
11902 -24.094233  121.869029 -24.089734  121.873528  0.0  0.0
11903 -26.555116  118.512871 -26.550617  118.517370  0.0  0.0
11904 -21.435401  118.476880 -21.430902  118.481379  0.0  0.0
11905 -25.317926  118.881779 -25.313427  118.886278  0.0  0.0
11906 -24.319177  121.158207 -24.314678  121.162706  0.0  0.0

[1500 rows x 6 columns]


In [ ]:
# download no mine images

#converts meters to Latitude-Longitude
ImageSize =500.0
ImageStep = float(ImageSize/111139)
print(ImageStep)
URL1 = 'https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Locate/ImageServer/exportImage?f=image&bbox='
Iterations = 1500
file_type = 'png'

# Enter download folder path

path = '/Mining/'
os.chdir(path)

#define df for record keeping
schema = {'ID' :  str,
          'Site_Code': str,
         'File_Name' : str,
         'Stage' : str,
         'SubType': str,
         'Type2' : str,    
         'Download_Date_Time' : str,
         'Latitude_Reference' : float,
         'Longtitude_Reference': float,
         'Latitude_1' : float,
         'Longitude_1' : float,
         'Latitude_2' : float,
         'Longitude_2' : float,
         'Format' : str,
         'Size': int,
         'Step':float}
image_df = pd.DataFrame(columns=schema).astype(schema)

for i,row in non_mine2.iloc[242:,:].iterrows():  
  
        LatRef = 0 #float(row['Latitude'])
        LongRef = 0 #float(row['Longitude'])
       

        
      #   Lat1 = LatRef - ImageStep/2
      #   Long1 = LongRef - ImageStep/2
      #   Lat2 = LatRef + ImageStep/2
      #   Long2 = LongRef + ImageStep/2

        Lat1 = row[0]
        Long1 = row[1] 
        Lat2 = row[2]
        Long2 = row[3]
        #URL2 = 'bbox='+ Lat1 + ''%2C' + Long1 + %2C' + Lat2 + ''%2C' + Long2
        URL = (URL1 + str(Long1) + ","+ str(Lat1) + ","+ str(Long2) + "," + str(Lat2)
              + '&format='+file_type +'&size=' + str(ImageSize) + ',' + str(ImageSize) + 'compressionQuality=100')

        print(URL)
        print("Image:"+ str(i) + '/' + str(Iterations) + "  "+ str(LatRef)+ ','+str(LongRef)+ ','+ str(Lat1) + ","+ str(Long1) + ","+ str(Lat2) + ","+ str(Long2))

        # Get image from API
        response = requests.get(URL)
        #print(response.text)

        #convert repsonse from binary to an image
        img = Image.open(BytesIO(response.content))

        image_id =datetime.now().strftime("%y%m%d%H%M%S")+str(i) 

        filename = str(i).zfill(12) +'.'+ file_type

        temp = pd.Series({'ID' :  i+1,
                         'Site_Code': 'Non Mine', #row['SiteCode'],
                         'File_Name' : filename,
                         'Stage' : 'Non Mine', #row['Stage'],
                         'SubType': 'Non Mine', #row['SubType'],
                         'Type2' : 'Non Mine', #row['Stage2'],    
                         'Download_Date_Time' : datetime.now().strftime("%y-%m-%d %H:%M:%S"),
                         'Latitude_Reference' : LatRef,
                         'Longtitude_Reference': LongRef,
                         'Latitude_1' : Lat1,
                         'Longitude_1' : Long1,
                         'Latitude_2' : Lat2,
                         'Longitude_2' : Long2,
                         'Format' : file_type,
                         'Size': ImageSize,
                         'Step': ImageStep})
        #print(temp)  
        image_df = pd.concat([image_df,temp.to_frame().T], ignore_index=True)

        #write image to file
        img.save(filename)
        centre = 250

        #  code to generate concentric squares on image to evaluate bounding box size:
        
        # for j in range(10,500,40): 
         
        #     shape = [(centre - j/2, centre - j/2), (centre + j/2, centre + j/2)] 

        #     img1 = ImageDraw.Draw(img)   
        #     img1.rectangle(shape, outline ='red', width=3) 
        #     img1.text((centre - j/2, centre - j/2), str(j))
        # filename = row['SubType'] + image_id +'_x.'+ file_type    
        # # filename = 'Costean'+ image_id +'_x.'+ file_type    

        # img.save(filename)

      #   label_file = str(11) + " " + str(row['x_center_norm'])+ " " + str(row['y_center_norm'])+ " " + str(row['width_norm'])+ " " + str(row['height_norm'])     #row['MineTypeID']) 
      #   filename = str(i).zfill(12) +'.txt'
      #   file1 = open(filename, 'w')
      #   file1.write(label_file)
      #   file1.close()    

#display(image_df)

select_df.to_csv("NewStyle_"+datetime.now().strftime("%y%m%d%H%M%S")+".csv")


0.004498870783433358
https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Locate/ImageServer/exportImage?f=image&bbox=120.30792071190284,-26.11422632919197,120.31241958268627,-26.10972745840854&format=png&size=500.0,500.0compressionQuality=100
Image:10649/1500  0,0,-26.11422632919197,120.30792071190284,-26.10972745840854,120.31241958268627
https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Locate/ImageServer/exportImage?f=image&bbox=122.54835836205416,-25.875786177670037,122.5528572328376,-25.8712873068866&format=png&size=500.0,500.0compressionQuality=100
Image:10650/1500  0,0,-25.875786177670037,122.54835836205416,-25.8712873068866,122.5528572328376
https://services.slip.wa.gov.au/public/rest/services/SLIP_Public_Services/Locate/ImageServer/exportImage?f=image&bbox=122.44038546325169,-23.054994196457663,122.44488433403512,-23.05049532567423&format=png&size=500.0,500.0compressionQuality=100
Image:10651/1500  0,0,-23.054994196457663,122.440385